In [1]:
import os
import time
import json

import pandas as pd
import ccxt

import yahoo_fin.stock_info as yahoo
from progressbar import progressbar

In [2]:
class GetPairs:
    ''' This class collects all names of the top trading pairs'''
    
    def __init__(self):
        self.path = "data/market_cap"

    def _clean_list(self, dirty_list):
        ''' Remove items in list that contain special characters or lowercase letters'''

        clean_list = dirty_list.copy()
    #     clean_list = [str(i) for i in clean_list]
        dirty_items = [".", ",", '/', "$", "%", "!", "@"]

        for item in dirty_list:
            for dirty_item in dirty_items:
                try:
                    if dirty_item in item:
                        clean_list.remove(item)
                except Exception as e:
                    pass

            if item[1].islower() or item[1].isnumeric():
                try:
                    clean_list.remove(item)
                except Exception as e:
                    pass

        return clean_list

    def get_historic_top_crypto(self):
        ''' Returns top crypto currencies by market cap since 2016 (dependent on the data in path)
            
            The historic crypto data is a bit dirty and has to be cleaned with the _clean_list
            function. 
            
            returns:
                pair_list_clean: list - List of historical trading pairs
        '''

        pair_set = set()

        for filename in os.listdir(self.path):
            try:
                df_temp = pd.read_csv(f"data/market_cap/{filename}")
                pair_set = set.union(pair_set, set(df_temp['Ticker']))
            except KeyError:
                pass

        pair_list_dirty = list(pair_set)
        pair_list_dirty = [str(i) for i in pair_list_dirty]
        pair_list_clean = self._clean_list(pair_list_dirty)

        return pair_list_clean
    
    def get_current_top_crypto(self):
        ''' Get current 100 crypto trading pairs'''
        # Automatische check toevoegen om te zien of de ticker al in de lijst staat.
        top100 = yahoo.get_top_crypto()

        return list(top100['Symbol'])
    

In [3]:
class RequestCryptoData:
    ''' This class requests data from the API of a specific exchange. 
        The API call will return the OHLCV data for a specified crypto trading pair (e.g. BTC/USD)
        
        params:
            pair: list - List trading pairs that need to be looked up
            timeframe: string - Time interval for data (e.g. 1m, 5m, 15m, 1h, 1d, 1wk, 1mo)
            start: int - Timestamp for the first record in the requested data
            req_number: int - Number of requests
            exchange: string - Name of the exchange (in lowercase)
            
    '''
    
    
    def __init__(self, pairs, timeframe, start, reduction, req_number, exchange):
        self.pairs = pairs
        self.timeframe = timeframe
        self.start = start
        self.reduction = reduction
        self.req_number = req_number
        self.exchange = exchange
        
        self.usdt_exchange = ['binance', 'huobi', 'kucoin', 'bitstamp'
                              'bittrex_usdt', 'gateio', 'okex'
                             ]
        
        self.columns = ["Date", "Open", "High", "Low", "Close", "Volume"]
#         self.check_df = pd.DataFrame(columns=self.columns)

        
    def __get_exchange(self):
        ''' Get exchange API function'''
        exchanges = {
            "binance": ccxt.binance(),
            "ftx": ccxt.ftx(),
            "kraken": ccxt.kraken(),
            "bitfinex": ccxt.bitfinex(), # max 100 rows
            "huobi": ccxt.huobi(), # 
            "gateio": ccxt.gateio(), # max 100 rows
            "kucoin": ccxt.kucoin(),
            "bittrex": ccxt.bittrex(),
            "bittrex_usdt": ccxt.bittrex(), #USD en USDT
            "coinbase": ccxt.coinbase(),
            "coinbasepro": ccxt.coinbasepro(),
            "bybit": ccxt.bybit(),
            "bitstamp": ccxt.bitstamp(),
            "gemini": ccxt.gemini(),
            "okex": ccxt.okex(),
            "hitbtc": ccxt.hitbtc()
                    }
        return exchanges[self.exchange]
    
    def __change_pairs(self):
        ''' Changes trading pair name from */USD to */USDT'''
        # Moet logica inbouwen om USDTT te voorkomen
        return [pair.replace("/USD", "/USDT") for pair in self.pairs]
        
    def _download_data(self, pair, timestamp):
        ''' Downloads and saves trading pair data'''
        exchange = self.__get_exchange()
        
        data = exchange.fetch_ohlcv(pair, self.timeframe, timestamp)
        df = pd.DataFrame.from_records(data, columns=self.columns)
        
        pair_name = pair.replace("/", "-")
        timestamp_name = int(timestamp/10000)
        self.save_directory = (f'data/request_{self.timeframe}'
                               f'/{self.exchange}_{pair_name}_{timestamp_name}.csv')

        return df
    
    def request_data(self):
        ''' Calls the _download_data function for a specified pair and timestamp. 
            
            If the request fails, the name of the failed pair is returned as the "failed" list. 
            This list can later be used to try and get the data from another exchange
            
            returns:
                success: list - A list of all the trading pairs that are downloaded
                failed: list - A list of all the trading pairs that failed to download
                timestamp_dict: dict - A dictionary of the earliest succesfully downloaded trading pair
        '''
        
        # Some exchanges only have */USDT trading pairs
        if self.exchange in self.usdt_exchange: 
            self.pairs = self.__change_pairs()
            
        pairs_slash = [x.replace("-", "/") for x in self.pairs]
        timestamp = self.start

        success = []
        failed = []
        timestamp_dict = {}
        error_dict = {}

        for pair in pairs_slash:
            try:
                for req in range(self.req_number):
                    time.sleep(10)
                    df = self._download_data(pair, timestamp)
#                     print(pair)
#                     print(df.head(5))
#                     if df.shape[0]<1:
#                         error = f"Maximum amount of avalaible {pair} data retrieved from {self.exchange}"
#                         error_dict[pair] = error
#                         failed.append(pair)
#                         break

                    df.to_csv(self.save_directory, index=False)

                    timestamp_dict[pair] = timestamp
                    timestamp -= self.reduction                    

    #             if df.values.all() != self.check_df.values.all(): # beetje omslagtig. Alternatief is om een self variable aan te maken
                success.append(pair)
                print(f"Request succesful: {pair}")
                    
            except ccxt.BadSymbol as bs:
                print(f"Could not download {pair}")
                print(bs)
                error_dict[pair] = str(bs)
                failed.append(pair)
                pass
            except ccxt.RateLimitExceeded as re:
                print(f"Too Many Requests {pair}")
                print(re)
                error_dict[pair] = str(re)
                pass
            except Exception as e:
                print(f"Error: {pair}")
                print(e)
                error_dict[pair] = str(e)
                pass

        return success, failed, timestamp_dict, error_dict

    # Logica inbouwen om duplicates te voorkomen

In [4]:
def search_exchanges(exchanges, timeframe, start, reduction, req_number):
    ''' Loops through list of exchanges to find data for all pairs'''
    
    pairs = GetPairs().get_current_top_crypto()
    logs = {}
    
    for exchange in progressbar(exchanges):
        print(f"Start data request: {exchange}")
        crypto = RequestCryptoData(pairs, timeframe, start, reduction, req_number, exchange)
        success, failed, final_timestamp, error_dict = crypto.request_data()
#         pairs = failed
        logs[exchange] = {
            "success": success, 
            "failed": failed, 
            "final_timestamp": final_timestamp,
            "error_dict": error_dict
                         }
        
        # Hier kan nog ingebouwd worden dat er een request wordt gemaakt
        # voor onvolledige data van een bepaalde ticker dmv een dubbele for loop
    
    return logs

def save_json(path, filename, logs):
    with open(f"{path}/{filename}.json", "w") as fp:
        json.dump(logs, fp, indent=4)

In [5]:
timeframe = "1h"
start = 1640386800000 #1640188800000
reduction = 356400000 #1796400000
req_number = 20


exchanges = ["binance","ftx","kraken","bitfinex","huobi",  
            "gateio", "kucoin","bittrex","bittrex_usdt", 
            "coinbase","coinbasepro", "bybit","bitstamp",
            "gemini","okex","hitbtc"]

logs = search_exchanges(exchanges, timeframe, start, reduction, req_number)

path = "data/request_1h"
filename = "logs"
save_json(path, filename, logs)

N/A% (0 of 16) |                         | Elapsed Time: 0:00:00 ETA:  --:--:--

Request succesful: BTC/USDT
Request succesful: ETH/USDT
Request succesful: BNB/USDT
Could not download USDTT/USDT
binance does not have market symbol USDTT/USDT
Request succesful: SOL/USDT
Could not download SOL1/USDT
binance does not have market symbol SOL1/USDT
Could not download HEX/USDT
binance does not have market symbol HEX/USDT
Request succesful: ADA/USDT
Request succesful: XRP/USDT
Could not download USDTC/USDT
binance does not have market symbol USDTC/USDT
Could not download LUNA1/USDT
binance does not have market symbol LUNA1/USDT
Request succesful: DOT/USDT
Request succesful: AVAX/USDT
Request succesful: DOGE/USDT
Could not download DOT1/USDT
binance does not have market symbol DOT1/USDT
Request succesful: SHIB/USDT
Request succesful: MATIC/USDT
Could not download CRO/USDT
binance does not have market symbol CRO/USDT
Request succesful: LTC/USDT
Could not download UNI1/USDT
binance does not have market symbol UNI1/USDT
Request succesful: LINK/USDT
Request succesful: ALGO/USDT

  6% (1 of 16) |#                | Elapsed Time: 4:17:49 ETA:  2 days, 16:27:23

Request succesful: QTUM/USDT
Request succesful: BTC/USD
Request succesful: ETH/USD
Request succesful: BNB/USD
Request succesful: USDT/USD
Request succesful: SOL/USD
Could not download SOL1/USD
ftx {"success":false,"error":"No such market: SOL1/USD "}
Could not download HEX/USD
ftx {"success":false,"error":"No such market: HEX/USD "}
Could not download ADA/USD
ftx {"success":false,"error":"No such market: ADA/USD "}
Request succesful: XRP/USD
Could not download USDC/USD
ftx {"success":false,"error":"No such market: USDC/USD "}
Could not download LUNA1/USD
ftx {"success":false,"error":"No such market: LUNA1/USD "}
Request succesful: DOT/USD
Request succesful: AVAX/USD
Request succesful: DOGE/USD
Could not download DOT1/USD
ftx {"success":false,"error":"No such market: DOT1/USD "}
Request succesful: SHIB/USD
Request succesful: MATIC/USD
Request succesful: CRO/USD
Request succesful: LTC/USD
Could not download UNI1/USD
ftx {"success":false,"error":"No such market: UNI1/USD "}
Request succes

 12% (2 of 16) |##                 | Elapsed Time: 6:34:14 ETA:  1 day, 7:49:49

...

Could not download QTUM/USD
gemini does not have market symbol QTUM/USD
Request succesful: BTC/USDT
Request succesful: ETH/USDT
Could not download BNB/USDT
okex does not have market symbol BNB/USDT
Could not download USDTT/USDT
okex does not have market symbol USDTT/USDT
Request succesful: SOL/USDT
Could not download SOL1/USDT
okex does not have market symbol SOL1/USDT
Could not download HEX/USDT
okex does not have market symbol HEX/USDT
Request succesful: ADA/USDT
Request succesful: XRP/USDT
Could not download USDTC/USDT
okex does not have market symbol USDTC/USDT
Could not download LUNA1/USDT
okex does not have market symbol LUNA1/USDT
Request succesful: DOT/USDT
Request succesful: AVAX/USDT
Request succesful: DOGE/USDT
Could not download DOT1/USDT
okex does not have market symbol DOT1/USDT
Request succesful: SHIB/USDT
Request succesful: MATIC/USDT
Request succesful: CRO/USDT
Request succesful: LTC/USDT
Could not download UNI1/USDT
okex does not have market symbol UNI1/USDT
Request s

 93% (15 of 16) |############### | Elapsed Time: 1 day, 14:53:25 ETA:   3:48:15

Request succesful: QTUM/USDT
Could not download BTC/USD
hitbtc does not have market symbol BTC/USD
Could not download ETH/USD
hitbtc does not have market symbol ETH/USD
Could not download BNB/USD
hitbtc does not have market symbol BNB/USD
Could not download USDT/USD
hitbtc does not have market symbol USDT/USD
Could not download SOL/USD
hitbtc does not have market symbol SOL/USD
Could not download SOL1/USD
hitbtc does not have market symbol SOL1/USD
Could not download HEX/USD
hitbtc does not have market symbol HEX/USD
Could not download ADA/USD
hitbtc does not have market symbol ADA/USD
Could not download XRP/USD
hitbtc does not have market symbol XRP/USD
Could not download USDC/USD
hitbtc does not have market symbol USDC/USD
Could not download LUNA1/USD
hitbtc does not have market symbol LUNA1/USD
Could not download DOT/USD
hitbtc does not have market symbol DOT/USD
Could not download AVAX/USD
hitbtc does not have market symbol AVAX/USD
Could not download DOGE/USD
hitbtc does not have 

100% (16 of 16) |#########| Elapsed Time: 1 day, 15:12:55 Time: 1 day, 15:12:55


Could not download QTUM/USD
hitbtc does not have market symbol QTUM/USD


In [6]:
logs

{'binance': {'success': ['BTC/USDT',
   'ETH/USDT',
   'BNB/USDT',
   'SOL/USDT',
   'ADA/USDT',
   'XRP/USDT',
   'DOT/USDT',
   'AVAX/USDT',
   'DOGE/USDT',
   'SHIB/USDT',
   'MATIC/USDT',
   'LTC/USDT',
   'LINK/USDT',
   'ALGO/USDT',
   'DAI/USDT',
   'BCH/USDT',
   'TRX/USDT',
   'XLM/USDT',
   'MANA/USDT',
   'AXS/USDT',
   'ATOM/USDT',
   'SAND/USDT',
   'VET/USDT',
   'FTT/USDT',
   'HBAR/USDT',
   'FTM/USDT',
   'ICP/USDT',
   'FIL/USDT',
   'EGLD/USDT',
   'ETC/USDT',
   'THETA/USDT',
   'HNT/USDT',
   'XTZ/USDT',
   'XMR/USDT',
   'AAVE/USDT',
   'EOS/USDT',
   'CAKE/USDT',
   'LRC/USDT',
   'STX/USDT',
   'FLOW/USDT',
   'BTT/USDT',
   'MKR/USDT',
   'ENJ/USDT',
   'KSM/USDT',
   'BSV/USDT',
   'QNT/USDT',
   'CRV/USDT',
   'ZEC/USDT',
   'AMP/USDT',
   'NEO/USDT',
   'AR/USDT',
   'RUNE/USDT',
   'BAT/USDT',
   'CHZ/USDT',
   'CELO/USDT',
   'WAVES/USDT',
   'DASH/USDT',
   'IOTX/USDT',
   'XEM/USDT',
   'YFI/USDT',
   '1INCH/USDT',
   'TFUEL/USDT',
   'ROSE/USDT',
   'IC